In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

## import data

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ["airplane", "automobile", "bird", "cat", "deer",
               "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# Let's look at a one image
IMG_INDEX = 20
plt.imshow(train_images[IMG_INDEX] ,cmap=plt.cm.binary)
plt.xlabel(class_names[train_labels[IMG_INDEX][0]])
plt.show()

## cnn architecture

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3),  # 32 filters & (3, 3) sample size of the filters 
          activation="relu",         # relu on the filters
          input_shape=(32, 32, 3)))  # image size (32, 32) & 3 dimensions due to rgb
model.add(layers.MaxPooling2D((2, 2)))
# after first layer, the next layers figure out on their own the input_shape
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))

model.summary()
# number of param: channels_in * kernel_width * kernel_height * channels_out + num_channels
# first layer: 3*3*3*32+32=896 & second layer: 32*3*3*64+64=18496

## adding dense layer

In [ ]:
model.add(layers.Flatten())  # last layer (4, 4, 64) flattened to a vector
model.add(layers.Dense(64, activation="relu"))  # hidden layer
model.add(layers.Dense(10))  # output layer

model.summary()

## training

In [17]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

history = model.fit(train_images, train_labels, epochs=4, validation_data=(test_images, test_labels))

Epoch 1/4
1563/1563 [==============================] - 64s 41ms/step - loss: 0.5391 - accuracy: 0.8096 - val_loss: 0.9193 - val_accuracy: 0.7063
Epoch 2/4
1563/1563 [==============================] - 64s 41ms/step - loss: 0.5129 - accuracy: 0.8190 - val_loss: 0.9571 - val_accuracy: 0.7058
Epoch 3/4
1563/1563 [==============================] - 64s 41ms/step - loss: 0.4823 - accuracy: 0.8301 - val_loss: 0.9449 - val_accuracy: 0.7079
Epoch 4/4
1563/1563 [==============================] - 63s 41ms/step - loss: 0.4563 - accuracy: 0.8389 - val_loss: 0.9956 - val_accuracy: 0.7161


## evaluate

In [ ]:
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label = "val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
plt.legend(loc="lower right")

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

## data augmentation

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# creates a data generator object that transforms images
datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode="nearest")

# pick an image to transform
test_img = train_images[20]
img = image.img_to_array(test_img)  # convert image to numpy array
img = img.reshape((1,) + img.shape)  # reshape image

i = 0

# this loops runs forever until we break, saving images to current directory with specified prefix
for batch in datagen.flow(img, save_prefix="test", save_format="jpeg"):
    plt.figure(i)
    plot = plt.imshow(image.img_to_array(batch[0]))
    i += 1
    if i > 4:  # show 4 images
        break

plt.show()
